# Extract inflation data using an API, calculate the inflation rate, and save it as a CSV file.

Reference Link: https://towardsdatascience.com/acquire-and-analyze-inflation-data-in-the-us-with-an-api-python-and-tableau-ae81944bcbb0

https://www.bls.gov/developers/api_signature_v2.htm

In [75]:
# pip install prettytable

In [77]:
import requests
import json
import pandas as pd

headers = {'Content-type': 'application/json'}
data = json.dumps({"seriesid": ['CUUR0000SA0'], "startyear": "2014", "endyear": "2025"})
response = requests.post('https://api.bls.gov/publicAPI/v2/timeseries/data/', data=data, headers=headers)
json_data = json.loads(response.text)

data_list = []  # This list will store the data dictionaries

for series in json_data['Results']['series']:
    seriesId = series['seriesID']
    for item in series['data']:
        year = item['year']
        period = item['period']
        value = item['value']
        footnotes = "".join([footnote['text'] for footnote in item['footnotes'] if footnote])  # Concatenate footnotes
        if 'M01' <= period <= 'M12':
            data_list.append({
                "series id": seriesId,
                "year": year,
                "period": period,
                "value": value,
                "footnotes": footnotes.rstrip(',')
            })

# Convert the list of dictionaries into a pandas DataFrame
df = pd.DataFrame(data_list)


In [78]:
df

,series id,year,period,value,footnotes
0,CUUR0000SA0,2023,M12,306.746,
1,CUUR0000SA0,2023,M11,307.051,
2,CUUR0000SA0,2023,M10,307.671,
3,CUUR0000SA0,2023,M09,307.789,
4,CUUR0000SA0,2023,M08,307.026,
...,...,...,...,...,...
115,CUUR0000SA0,2014,M05,237.900,
116,CUUR0000SA0,2014,M04,237.072,
117,CUUR0000SA0,2014,M03,236.293,
118,CUUR0000SA0,2014,M02,234.781,


In [79]:
df = df.drop('series id', axis=1)
df = df.drop('footnotes', axis=1)
df['period'] = df['period'].str.slice(1,4)

df['year'] = df['year'].astype(int)
df['period'] = df['period'].astype(int)
df['value'] = df['value'].astype(float)
df.reset_index()

,index,year,period,value
0,0,2023,12,306.746
1,1,2023,11,307.051
2,2,2023,10,307.671
3,3,2023,9,307.789
4,4,2023,8,307.026
...,...,...,...,...
115,115,2014,5,237.900
116,116,2014,4,237.072
117,117,2014,3,236.293
118,118,2014,2,234.781


In [80]:
df.sort_values(by=['year','period'])

,year,period,value
119,2014,1,233.916
118,2014,2,234.781
117,2014,3,236.293
116,2014,4,237.072
115,2014,5,237.900
...,...,...,...
4,2023,8,307.026
3,2023,9,307.789
2,2023,10,307.671
1,2023,11,307.051


In [85]:
# Sort the DataFrame by Year and Month if needed
# This assumes 'Month' is in full month name format. Adjust the sort logic if your data format differs.
df['Date'] = pd.to_datetime(df['year'].astype(str) + ' ' + df['period'].astype(str))
df = df.sort_values(by='Date')

# Calculate the month-over-month percentage change in CPI
df['Inflation Rate (%)'] = df['value'].pct_change() * 100
df['Inflation_Rate'] = df['Inflation Rate (%)']
df['Month'] = df['period']

In [86]:
df

,year,period,value,Date,Inflation Rate (%),Inflation_Rate,Month
119,2014,1,233.916,2014-01-01,NaN,NaN,1
118,2014,2,234.781,2014-02-01,0.369791,0.369791,2
117,2014,3,236.293,2014-03-01,0.644004,0.644004,3
116,2014,4,237.072,2014-04-01,0.329675,0.329675,4
115,2014,5,237.900,2014-05-01,0.349261,0.349261,5
...,...,...,...,...,...,...,...
4,2023,8,307.026,2023-08-01,0.436716,0.436716,8
3,2023,9,307.789,2023-09-01,0.248513,0.248513,9
2,2023,10,307.671,2023-10-01,-0.038338,-0.038338,10
1,2023,11,307.051,2023-11-01,-0.201514,-0.201514,11


In [87]:
df.to_csv('./inflation_data.csv', index=False)